In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 8
fig_height = 4
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/diego/local_repos/AGEC652_2024/slides/lecture_6_1"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]


[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


In [2]:
#| include: false
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("JuMP")
# Pkg.add("Ipopt")
# Pkg.add("Statistics")
# Pkg.add("Plots")
# Pkg.add("ForwardDiff")
# Pkg.add("LinearAlgebra")

  Activating new project at `~/local_repos/AGEC652_2024/slides/lecture_6_1`


  No Changes to `~/local_repos/AGEC652_2024/slides/lecture_6_1/Project.toml`
  No Changes to `~/local_repos/AGEC652_2024/slides/lecture_6_1/Manifest.toml`


In [3]:
function log_likelihood(θ) 
  β0, β1, σ = θ # Unpack parameters
  return -n/2*log(2*pi) - n/2*log(σ^2) - (1/(2*σ^2)) * sum((y .- β0 .- β1 .* x).^2)
end

log_likelihood (generic function with 1 method)

In [4]:
using JuMP, Ipopt, Statistics

# Generate some synthetic data for illustration
n = 1000  # Number of observations
x = rand(n) * 10  # Independent variable: random uniform [0,10]
β0_true = 2.0  # True intercept
β1_true = 3.0  # True slope
σ_true = 1.5  # True standard deviation of the errors
ε = randn(n) * σ_true  # Normally distributed errors
y = β0_true .+ β1_true .* x .+ ε  # Dependent variable

[ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]


[ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]


1000-element Vector{Float64}:
  3.709130909626828
  4.358157418145898
 22.719839584280113
  7.207996637417205
 25.627616295122998
  8.195313133149636
 19.708693736146433
  2.251019401377114
 16.807801515978028
 11.787769533281057
  8.014130941988618
 25.307247717114297
 14.176323946466058
  ⋮
 12.298406774933483
 13.750613022987906
 30.256221684296833
 26.04527755565141
 16.318591269235245
 14.040926024896885
 29.22609478051674
 11.105921708667193
  4.161973917714973
  4.792055871882438
 26.081272486275004
  3.1345723071247478

In [5]:
# Define the model
mle_model = Model(Ipopt.Optimizer);
@variable(mle_model, β0);
@variable(mle_model, β1);
@variable(mle_model, σ >= 0.0001);  # Avoid division by zero in the log-likelihood

# Set the objective to maximize the log-likelihood
@objective(mle_model, Max, log_likelihood([β0, β1, σ]))

(-918.9385332046727 - (500.0 * log(σ²))) - ((1.0 / (2 σ²)) * (1000 β0² + 10317.789033264782 β1*β0 + 34614.43894282611 β1² - 35032.88593005477 β0 - 229034.2838654797 β1 + 381923.42476662307))

In [6]:
# Solve the model
optimize!(mle_model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

In [7]:
# Retrieve the estimated parameters
β0_hat = value(β0);
β1_hat = value(β1);
σ_hat = value(σ);

println("Estimated parameters:")
println("β0: ", β0_hat)
println("β1: ", β1_hat)
println("σ: ", σ_hat)

Estimated parameters:


β0: 1.9424389363062382
β1: 3.0188645994647034
σ: 1.478870543971837


In [8]:
using ForwardDiff, LinearAlgebra
θ_hat = [β0_hat, β1_hat, σ_hat]
Im = ForwardDiff.hessian(log_likelihood, θ_hat)

3×3 Matrix{Float64}:
  -457.235        -2358.83          -5.38376e-8
 -2358.83        -15826.9           -3.12965e-7
    -5.38376e-8      -3.12965e-7  -914.47

In [9]:
V = inv(Im);
SEs = sqrt.(diag(-V))

3-element Vector{Float64}:
 0.09727631895255799
 0.016534020803202425
 0.03306855037191697

In [10]:
using DataFrames, Distributions
df = DataFrame(
  Coefficient = ["beta_0", "beta_1", "sigma"],
  Estimate = θ_hat,
  StdError = SEs,
  CI_lower = θ_hat .+ quantile(Normal(), 0.025) .* SEs,
  CI_upper = θ_hat .+ quantile(Normal(), 0.975) .* SEs
)

println(df)

[ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]


[ Info: Precompiling DistributionsTestExt [ffbe0ea5-a612-5ff7-aaf5-cac02eef3019]


3×5 DataFrame
 Row │ Coefficient  Estimate  StdError   CI_lower  CI_upper 
     │ String       Float64   Float64    Float64   Float64  
─────┼──────────────────────────────────────────────────────
   1 │ beta_0        1.94244  0.0972763   1.75178   2.1331
   2 │ beta_1        3.01886  0.016534    2.98646   3.05127
   3 │ sigma         1.47887  0.0330686   1.41406   1.54368


In [11]:
# Define the model
mle_model_restr = Model(Ipopt.Optimizer);
set_silent(mle_model_restr)
@variable(mle_model_restr, β0);
@variable(mle_model_restr, σ >= 0.0001);  # Avoid division by zero in the log-likelihood

# Note β0 being used twice!
@objective(mle_model_restr, Max, log_likelihood([β0, β0, σ]));

optimize!(mle_model_restr);

In [12]:
β0_hat_R = value(β0);
σ_hat_R = value(σ);

println("Estimated parameters:")
println("β0=β1: ", β0_hat)
println("σ: ", σ_hat)

Estimated parameters:
β0=β1: 1.9424389363062382
σ: 1.478870543971837


In [13]:
l_U = log_likelihood([β0_hat, β1_hat, σ_hat]);
l_R = log_likelihood([β0_hat_R, β0_hat_R, σ_hat_R]);

In [14]:
LR_stat = 2 * (l_U - l_R)

88.26453606487894

In [15]:
crit_value_lower = quantile(Chisq(1), 0.025);
crit_value_upper = quantile(Chisq(1), 0.975);
println("Test interval: $crit_value_lower -- $crit_value_upper . Test statistic: $LR_stat")

Test interval: 0.0009820691171752564 -- 5.023886187314887 . Test statistic: 88.26453606487894


In [16]:
p_value = 1 - cdf(Chisq(1), LR_stat)

0.0